In [ ]:
from openai import OpenAI

In [ ]:
#ollama_url = "http://localhost:11434/v1"
#ollama = OpenAI(api_key="ollama", base_url=ollama_url)
openai = OpenAI()

In [ ]:
class Player:
    def __init__(self, name, traits, relations):
        self.name = name
        self.traits = traits  # Dict of trait descriptions
        self.relations = relations  # Dict mapping player names to Relation objects

    def get_personality_prompt(self):
        """Generate a personality description for the LLM"""
        traits_desc = "\n".join([f"- {key}: {value}" for key, value in self.traits.items()])
        relations_desc = "\n".join([
            f"- {name}: {rel.value}/5 ({'friend' if rel.value > 0 else 'enemy' if rel.value < 0 else 'neutral'})"
            for name, rel in self.relations.items()
        ])
        
        return f"""You are {self.name}, a person with the following personality traits:
{traits_desc}

Your current relationships with others:
{relations_desc}

Play this character authentically. Your personality traits should influence how you communicate and make decisions. Your relationships affect how receptive you are to others' ideas."""

    def respond(self, prompt, conversation_history):
        """Generate a response based on personality and current situation"""
        messages = [
            {"role": "system", "content": self.get_personality_prompt()},
        ]
        
        # Add conversation history
        messages.extend(conversation_history)
        
        # Add current prompt
        messages.append({"role": "user", "content": prompt})
        
        response = openai.chat.completions.create(
            model="gpt-4o-mini",
            messages=messages,
            temperature=0.8
        )
        return response.choices[0].message.content

    def update_relation(self, other_player_name, change):
        """Update relationship value, clamped between -5 and +5"""
        if other_player_name in self.relations:
            self.relations[other_player_name].value = max(-5, min(5, 
                self.relations[other_player_name].value + change))


In [ ]:
class Relation:
    def __init__(self, other_player, value):
        self.other_player = other_player
        self.value = value

In [ ]:
# Run the simulation
scenario = "The group needs to decide where to eat lunch. Everyone has different preferences and dietary restrictions."

sim = Simulation(players, scenario)
sim.run()


In [ ]:
# Create 4 players with dysfunctional traits

# Player 1: The Stubborn One
alice_relations = {}

# Player 2: The Passive-Aggressive One  
bob_relations = {}

# Player 3: The Drama Queen/King
carol_relations = {}

# Player 4: The Know-It-All
dave_relations = {}

# Initialize players first (without relations)
alice = Player(
    name="Alice",
    traits={
        "Stubbornness": "Extremely rigid in opinions, refuses to compromise",
        "Directness": "Blunt and confrontational in communication",
        "Confidence": "Believes their way is always the right way"
    },
    relations=alice_relations
)

bob = Player(
    name="Bob",
    traits={
        "Passive-Aggressiveness": "Never directly states disagreement, uses sarcasm and backhanded compliments",
        "Avoidance": "Dislikes confrontation, agrees outwardly but undermines decisions",
        "Resentment": "Holds grudges and brings up past issues"
    },
    relations=bob_relations
)

carol = Player(
    name="Carol",
    traits={
        "Dramatic": "Blows everything out of proportion, sees slights where none exist",
        "Emotional": "Takes everything personally, reactions are extreme",
        "Attention-Seeking": "Needs to be the center of discussion, dismisses others' concerns"
    },
    relations=carol_relations
)

dave = Player(
    name="Dave",
    traits={
        "Superiority": "Acts intellectually superior, condescending to others",
        "Dismissiveness": "Quickly dismisses others' ideas as inferior",
        "Argumentative": "Must prove others wrong, debates every point"
    },
    relations=dave_relations
)

# Now set up the relationships (starting slightly negative for dysfunction)
alice_relations.update({
    "Bob": Relation(bob, -1),
    "Carol": Relation(carol, 0),
    "Dave": Relation(dave, -2)
})

bob_relations.update({
    "Alice": Relation(alice, -1),
    "Carol": Relation(carol, 1),
    "Dave": Relation(dave, 0)
})

carol_relations.update({
    "Alice": Relation(alice, 0),
    "Bob": Relation(bob, 1),
    "Dave": Relation(dave, -1)
})

dave_relations.update({
    "Alice": Relation(alice, -2),
    "Bob": Relation(bob, 0),
    "Carol": Relation(carol, -1)
})

players = [alice, bob, carol, dave]
print("Players initialized!")
print("\nStarting relationships:")
for player in players:
    print(f"\n{player.name}:")
    for other_name, relation in player.relations.items():
        print(f"  → {other_name}: {relation.value}/5")


In [ ]:
class Simulation:
    def __init__(self, players, scenario):
        self.players = players
        self.scenario = scenario
        self.conversation_history = []
        self.round_number = 0
        self.max_rounds = 15
        
    def check_win_condition(self):
        """Check if all players have reached consensus"""
        prompt = """Based on the conversation so far, have all participants reached a clear agreement? 
        Respond with only 'YES' or 'NO'."""
        
        response = openai.chat.completions.create(
            model="gpt-4o-mini",
            messages=[
                {"role": "system", "content": "You are evaluating whether a group has reached consensus."},
                {"role": "user", "content": f"Conversation:\n{self._format_history()}\n\n{prompt}"}
            ],
            temperature=0.3
        )
        
        return "YES" in response.choices[0].message.content.upper()
    
    def check_lose_condition(self):
        """Check if any player has reached -5 with another"""
        for player in self.players:
            for other_name, relation in player.relations.items():
                if relation.value <= -5:
                    return True, f"{player.name} has reached maximum hatred (-5) toward {other_name}"
        return False, None
    
    def _format_history(self):
        """Format conversation history for display"""
        return "\n".join([f"{msg['role']}: {msg['content']}" for msg in self.conversation_history])
    
    def evaluate_interaction(self, speaker_name, response):
        """Use LLM to evaluate how the response affects relationships"""
        for player in self.players:
            if player.name == speaker_name:
                continue
                
            eval_prompt = f"""You are {player.name}. {speaker_name} just said: "{response}"

Based on your personality and current relationship with {speaker_name} ({player.relations[speaker_name].value}/5), how does this statement make you feel?

Consider:
- Did they respect your opinion?
- Did they dismiss or insult you?
- Did they show understanding?
- Did they compromise or stay rigid?

Respond with a number from -2 to +2 indicating the change in your relationship:
-2: Very negative, insulting or dismissive
-1: Slightly negative, inconsiderate
0: Neutral, no significant impact
+1: Slightly positive, respectful
+2: Very positive, showing empathy and understanding

Respond with ONLY the number."""

            eval_response = openai.chat.completions.create(
                model="gpt-4o-mini",
                messages=[
                    {"role": "system", "content": player.get_personality_prompt()},
                    {"role": "user", "content": eval_prompt}
                ],
                temperature=0.5
            )
            
            try:
                change = int(eval_response.choices[0].message.content.strip())
                change = max(-2, min(2, change))  # Clamp to -2 to +2
                player.update_relation(speaker_name, change)
                
                if change != 0:
                    print(f"  → {player.name}'s feeling toward {speaker_name}: {change:+d} (now {player.relations[speaker_name].value}/5)")
            except:
                pass  # If we can't parse the evaluation, skip it
    
    def run_round(self):
        """Run one round where each player speaks"""
        self.round_number += 1
        print(f"\n{'='*60}")
        print(f"ROUND {self.round_number}")
        print(f"{'='*60}\n")
        
        for player in self.players:
            prompt = f"""Scenario: {self.scenario}

This is round {self.round_number}. Respond to the situation and others' comments. Be authentic to your personality.
Keep your response concise (2-3 sentences max)."""
            
            response = player.respond(prompt, self.conversation_history)
            
            print(f"{player.name}: {response}\n")
            
            # Add to history
            self.conversation_history.append({
                "role": "assistant",
                "content": f"{player.name}: {response}"
            })
            
            # Evaluate how this affects relationships
            self.evaluate_interaction(player.name, response)
            
            # Check lose condition after each player speaks
            lost, reason = self.check_lose_condition()
            if lost:
                return "LOSE", reason
        
        # Check win condition after the round
        if self.check_win_condition():
            return "WIN", "All players reached consensus!"
        
        if self.round_number >= self.max_rounds:
            return "TIMEOUT", "Maximum rounds reached without consensus"
        
        return "CONTINUE", None
    
    def run(self):
        """Run the full simulation"""
        print(f"\n{'#'*60}")
        print(f"SOCIAL DYSFUNCTION SIMULATION")
        print(f"{'#'*60}\n")
        print(f"Scenario: {self.scenario}\n")
        
        print("Players:")
        for player in self.players:
            print(f"\n{player.name}:")
            for trait, desc in player.traits.items():
                print(f"  • {trait}: {desc}")
        
        print(f"\n{'#'*60}\n")
        
        while True:
            status, message = self.run_round()
            
            if status != "CONTINUE":
                print(f"\n{'='*60}")
                print(f"GAME OVER: {status}")
                print(f"{'='*60}")
                print(f"\n{message}\n")
                
                # Show final relationships
                print("Final Relationships:")
                for player in self.players:
                    print(f"\n{player.name}:")
                    for other_name, relation in player.relations.items():
                        print(f"  → {other_name}: {relation.value}/5")
                
                break
